In [1]:
from keras.models import Sequential      
from keras.layers import Dense, Flatten  
from collections import deque            
import numpy as np
from keras.layers import Dense,Dropout
from tqdm import tqdm
from keras import optimizers
import random
from collections import Counter
import gym
import time

Using TensorFlow backend.


In [2]:
env = gym.make('CartPole-v0')

In [3]:
model = Sequential()
model.add(Dense(50, input_shape=(4,),activation='relu'))
model.add(Dense(256,kernel_initializer='normal', activation='relu'))
Dropout(0.5)
model.add(Dense(128,kernel_initializer='normal', activation='relu'))
Dropout(0.5)
model.add(Dense(512,kernel_initializer='normal', activation='relu'))
Dropout(0.5)
model.add(Dense(128,kernel_initializer='normal', activation='relu'))
Dropout(0.5)
model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))    

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                250       
_________________________________________________________________
dense_2 (Dense)              (None, 256)               13056     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 512)               66048     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 178,043
Trainable params: 178,043
Non-trainable params: 0
________________________________________________

In [5]:
observetime = 500
epsilon = 1.0
score_requirement = 50

In [6]:
def epsilon_update(epsilon):
    epsilon_decay = 0.95
    lowest_epsilon = 0.3
    if epsilon > lowest_epsilon:
        epsilon *= epsilon_decay
        
    return epsilon

In [7]:
def population(initial_games):  
    accepted_scores=[]
    training_data=[]
    scores=[]
    
    for _ in tqdm(range(initial_games)):
        observation = env.reset()
        obs = observation
        state = obs
        score=0
        game_memory=[]
        prev_observation=[]
        epsilon = 1.0
        for _ in range(observetime):           
            env.render()
            if np.random.rand() <= epsilon:
                action = random.randint(0,1)
            else:
                state = np.squeeze(state).reshape(1,4)
                Q = model.predict(state) 
                action = np.int(np.round(Q))
            observation_new, reward, done, info = env.step(action)     
            obs_new = observation_new          
            state_new = obs_new     
            if len(prev_observation)>0:
                game_memory.append([prev_observation,action])
            state = state_new         
            prev_observation = state_new
            score+=1
            epsilon = epsilon_update(epsilon)
                
            if done:
                #print(epsilon)
                break
        
        #if score>=score_requirement:
            accepted_scores.append(scores)
            for data in game_memory:
                if data[1]==1:
                    output=[1]
                else: 
                    output=[0]
                
                training_data.append([data[0],output])
                                
                
        env.reset()
        scores.append(score)
    
    np.save('mem.npz',training_data)
    #print(accepted_scores)
    #print(scores)
    #print(len(accepted_scores))
    print("Avg accepted score: ",np.mean(accepted_scores))
    print("Median accepted score: ",np.median(accepted_scores))
    env.close()     
    
    return training_data

In [8]:
training_data = population(2500)

100%|██████████| 2500/2500 [05:18<00:00,  7.84it/s]


Avg accepted score:  18.2544
Median accepted score:  17.0


In [9]:
X = np.array([i[0] for i in training_data])
y = [i[1] for i in training_data]

In [10]:
X.ndim

2

In [12]:
X = np.array(X)

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(np.array(X),np.array(y),batch_size=100,epochs=5)

Epoch 1/5
403245/403245 [==============================] - ETA: 26:08 - loss: 0.6928 - accuracy: 0.620 - ETA: 1:28 - loss: 0.6670 - accuracy: 0.633 - ETA: 45s - loss: 0.6636 - accuracy: 0.6247 - ETA: 33s - loss: 0.6596 - accuracy: 0.624 - ETA: 27s - loss: 0.6605 - accuracy: 0.619 - ETA: 23s - loss: 0.6582 - accuracy: 0.622 - ETA: 20s - loss: 0.6553 - accuracy: 0.621 - ETA: 19s - loss: 0.6546 - accuracy: 0.620 - ETA: 17s - loss: 0.6552 - accuracy: 0.618 - ETA: 16s - loss: 0.6549 - accuracy: 0.619 - ETA: 15s - loss: 0.6536 - accuracy: 0.620 - ETA: 14s - loss: 0.6547 - accuracy: 0.617 - ETA: 14s - loss: 0.6539 - accuracy: 0.619 - ETA: 13s - loss: 0.6537 - accuracy: 0.618 - ETA: 13s - loss: 0.6538 - accuracy: 0.618 - ETA: 12s - loss: 0.6541 - accuracy: 0.617 - ETA: 12s - loss: 0.6538 - accuracy: 0.617 - ETA: 12s - loss: 0.6538 - accuracy: 0.617 - ETA: 12s - loss: 0.6539 - accuracy: 0.618 - ETA: 11s - loss: 0.6542 - accuracy: 0.617 - ETA: 11s - loss: 0.6544 - accuracy: 0.617 - ETA: 11s - lo

403245/403245 [==============================] - ETA: 48s - loss: 0.6286 - accuracy: 0.620 - ETA: 10s - loss: 0.6350 - accuracy: 0.617 - ETA: 9s - loss: 0.6381 - accuracy: 0.623 - ETA: 9s - loss: 0.6433 - accuracy: 0.61 - ETA: 9s - loss: 0.6449 - accuracy: 0.61 - ETA: 9s - loss: 0.6457 - accuracy: 0.61 - ETA: 9s - loss: 0.6470 - accuracy: 0.61 - ETA: 9s - loss: 0.6468 - accuracy: 0.61 - ETA: 9s - loss: 0.6462 - accuracy: 0.61 - ETA: 9s - loss: 0.6470 - accuracy: 0.61 - ETA: 9s - loss: 0.6479 - accuracy: 0.61 - ETA: 9s - loss: 0.6485 - accuracy: 0.61 - ETA: 9s - loss: 0.6476 - accuracy: 0.61 - ETA: 9s - loss: 0.6474 - accuracy: 0.61 - ETA: 9s - loss: 0.6479 - accuracy: 0.61 - ETA: 9s - loss: 0.6474 - accuracy: 0.61 - ETA: 9s - loss: 0.6467 - accuracy: 0.61 - ETA: 9s - loss: 0.6465 - accuracy: 0.61 - ETA: 9s - loss: 0.6462 - accuracy: 0.61 - ETA: 9s - loss: 0.6462 - accuracy: 0.61 - ETA: 9s - loss: 0.6459 - accuracy: 0.61 - ETA: 9s - loss: 0.6463 - accuracy: 0.61 - ETA: 9s - loss: 0.6464

403245/403245 [==============================] - ETA: 48s - loss: 0.6200 - accuracy: 0.630 - ETA: 11s - loss: 0.6410 - accuracy: 0.623 - ETA: 10s - loss: 0.6406 - accuracy: 0.623 - ETA: 9s - loss: 0.6459 - accuracy: 0.612 - ETA: 9s - loss: 0.6472 - accuracy: 0.61 - ETA: 9s - loss: 0.6465 - accuracy: 0.61 - ETA: 9s - loss: 0.6452 - accuracy: 0.61 - ETA: 9s - loss: 0.6453 - accuracy: 0.61 - ETA: 9s - loss: 0.6438 - accuracy: 0.61 - ETA: 9s - loss: 0.6433 - accuracy: 0.61 - ETA: 9s - loss: 0.6429 - accuracy: 0.61 - ETA: 9s - loss: 0.6440 - accuracy: 0.61 - ETA: 9s - loss: 0.6431 - accuracy: 0.61 - ETA: 9s - loss: 0.6435 - accuracy: 0.61 - ETA: 9s - loss: 0.6439 - accuracy: 0.61 - ETA: 9s - loss: 0.6447 - accuracy: 0.61 - ETA: 9s - loss: 0.6440 - accuracy: 0.61 - ETA: 9s - loss: 0.6442 - accuracy: 0.61 - ETA: 9s - loss: 0.6444 - accuracy: 0.61 - ETA: 8s - loss: 0.6445 - accuracy: 0.61 - ETA: 8s - loss: 0.6445 - accuracy: 0.61 - ETA: 8s - loss: 0.6441 - accuracy: 0.61 - ETA: 8s - loss: 0.64

403245/403245 [==============================] - ETA: 48s - loss: 0.6891 - accuracy: 0.570 - ETA: 11s - loss: 0.6514 - accuracy: 0.609 - ETA: 10s - loss: 0.6494 - accuracy: 0.609 - ETA: 9s - loss: 0.6491 - accuracy: 0.611 - ETA: 9s - loss: 0.6475 - accuracy: 0.61 - ETA: 9s - loss: 0.6463 - accuracy: 0.61 - ETA: 9s - loss: 0.6463 - accuracy: 0.61 - ETA: 9s - loss: 0.6463 - accuracy: 0.61 - ETA: 9s - loss: 0.6458 - accuracy: 0.61 - ETA: 9s - loss: 0.6463 - accuracy: 0.61 - ETA: 8s - loss: 0.6466 - accuracy: 0.61 - ETA: 8s - loss: 0.6465 - accuracy: 0.61 - ETA: 8s - loss: 0.6466 - accuracy: 0.61 - ETA: 8s - loss: 0.6462 - accuracy: 0.61 - ETA: 8s - loss: 0.6464 - accuracy: 0.61 - ETA: 8s - loss: 0.6459 - accuracy: 0.61 - ETA: 8s - loss: 0.6453 - accuracy: 0.61 - ETA: 8s - loss: 0.6451 - accuracy: 0.61 - ETA: 8s - loss: 0.6454 - accuracy: 0.61 - ETA: 8s - loss: 0.6453 - accuracy: 0.61 - ETA: 8s - loss: 0.6455 - accuracy: 0.61 - ETA: 8s - loss: 0.6453 - accuracy: 0.61 - ETA: 8s - loss: 0.64

403245/403245 [==============================] - ETA: 52s - loss: 0.6784 - accuracy: 0.550 - ETA: 11s - loss: 0.6440 - accuracy: 0.623 - ETA: 10s - loss: 0.6428 - accuracy: 0.612 - ETA: 10s - loss: 0.6451 - accuracy: 0.615 - ETA: 9s - loss: 0.6472 - accuracy: 0.614 - ETA: 9s - loss: 0.6459 - accuracy: 0.61 - ETA: 9s - loss: 0.6468 - accuracy: 0.61 - ETA: 9s - loss: 0.6452 - accuracy: 0.61 - ETA: 9s - loss: 0.6453 - accuracy: 0.61 - ETA: 9s - loss: 0.6455 - accuracy: 0.61 - ETA: 9s - loss: 0.6452 - accuracy: 0.61 - ETA: 9s - loss: 0.6448 - accuracy: 0.61 - ETA: 9s - loss: 0.6441 - accuracy: 0.61 - ETA: 9s - loss: 0.6451 - accuracy: 0.61 - ETA: 9s - loss: 0.6449 - accuracy: 0.61 - ETA: 8s - loss: 0.6454 - accuracy: 0.61 - ETA: 8s - loss: 0.6451 - accuracy: 0.61 - ETA: 8s - loss: 0.6446 - accuracy: 0.61 - ETA: 8s - loss: 0.6440 - accuracy: 0.61 - ETA: 8s - loss: 0.6434 - accuracy: 0.61 - ETA: 8s - loss: 0.6434 - accuracy: 0.61 - ETA: 8s - loss: 0.6437 - accuracy: 0.61 - ETA: 8s - loss: 0.

In [19]:
def play():                               
    env = gym.make('CartPole-v0') 
    observation = env.reset()
    obs = observation
    state = obs
    done = False
    tot_reward = 0.0
    while not done:
        env.render()                    
        state = np.squeeze(state).reshape(1,4)
        Q = model.predict(state)        
        action = np.int(np.round(Q))
        observation, reward, done, info = env.step(action)
        obs = observation
        state = obs    
        tot_reward += reward
        time.sleep(0.05)
    env.close()
    print('done')

In [20]:
play()

done
